<a href="https://colab.research.google.com/github/AlbertoMontanelli/Machine-Learning/blob/class_unit/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sympy import symbols, diff

# dobbiamo scrivere anche le derivate
# dobbiamo rivedere le funzioni e le derivate in base di net e non di input, weight e bias
# dobbiamo capire come aggiornare i bias
# dobbiamo capire come funziona il minibatch
# dobbiamo capire che funzioni di attivazione usare e quali derivate
# da iniziare a fare successivamente: cross validation, test vs training error, ricerca di iperparametri (grid search, n layer, n unit,
# learning rule), nr epochs/early stopping, tikhonov regularization, momentum, adaline e altre novelties

def sigmoid(input_array, weight_array, bias):
    return 1 / (1 + np.exp(-(np.dot(input_array, weight_array) + bias)))

def tanh(input_array, weight_array, bias):
    return np.tanh(np.dot(input_array, weight_array) + bias)

def softmax(input_array, weight_array, bias):
    return np.exp(np.dot(input_array, weight_array) + bias) / np.sum(np.exp(np.dot(input_array, weight_array) + bias))

def softplus(input_array, weight_array, bias):
    return np.log(1 + np.exp(np.dot(input_array, weight_array) + bias))

def linear(input_array, weight_array, bias):
    return np.dot(input_array, weight_array) + bias

def d_linear(input_array, weight_array):
    return 1

def ReLU(input_array, weight_array, bias):
    return np.maximum(np.dot(input_array,weight_array) + bias, 0)

def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred)**2) # in output ho un numero

def mean_squared_error_derivative(y_true, y_pred):
    return 2 * (y_true - y_pred) / y_true.size # in output ho un array grande come il numero di nauroni del layer

class Layer:
    def __init__(self, input_size, output_size, activation_function, activation_derivative):
        #input_size: number of previous units
        #output_size: number of actual units
        self.weights = np.random.uniform(low=-1/np.sqrt(input_size), high=1/np.sqrt(input_size), size=(input_size, output_size)) # random from uniform distribution in [-1/a, 1/a] with a=sqrt(number of neuron of previous layer)
        self.biases = np.zeros((1, output_size)) # array 1D perché lo stesso per tutti gli esempi, identifica un neurone. must be 0
        self.activation_function = activation_function
        self.activation_derivative = activation_derivative

    # funzione che ci permette di calcolare gli output del layer. PRende come input l'output del layer precedente
    def forward(self, input_array):
        self.input = input_array # array 1D of previous unit or matrix with number of row = number of examples
        self.output = self.activation_function(self.input, self.weights, self.biases) #f(net)
        return self.output

    def backward(self, d_Ep, learning_rate):
        # d_Ep = target - output solo per output layer, il resto delle volte d_Ep = sum_delta_weights
        delta = d_Ep * self.activation_derivative(self.input, self.weights)
        self.weights += learning_rate * np.dot(self.input.T, delta)
        #VANNO AGGIORNATI ANCHE I BIAS
        sum_delta_weights = np.dot(delta, self.weights.T)
        return sum_delta_weights

class NeuralNetwork:
    def __init__(self):
        self.layers = [] # questa riga serve ad inizializzare una lista vuota. tutti i layers che verranno creati verranno aggiunti a questa lista

    def add_layer(self, layer):
        self.layers.append(layer)

    # ora dobbiamo fare la backprop per tutti i layer
    def forward(self, input): # questo input array sono proprio i dati che abbiamo a disposizione
        for layer in self.layers:
            input = layer.forward(input) # restituisce l'array di output e lo inserisco in input così da usarlo per il layer dopo
        return input

    def backward(self, d_Ep, learning_rate):
        for layer in reversed(self.layers): # così attraversa la lista in ordine inverso. il gradiente dell'errore propaga all'inverso
            d_Ep = layer.backward(d_Ep, learning_rate)

    # x: dataset, examples x features
    # epochs:  quante volte passo attraverso la rete neurale. Lo scelgo io??
    # loss function and derivative: MSE

    def train(self, x_train, target, epochs, learning_rate, loss_function, loss_function_derivative):
        for epoch in range(epochs):
            # Forward propagation
            predictions = self.forward(x_train) # ritorna gli output dell'ultimo layer

            # Compute loss and loss gradient for backward function
            loss = loss_function(target, predictions)
            loss_gradient = loss_function_derivative(target, predictions)

            # Backward propagation
            self.backward(loss_gradient, learning_rate)

            # Print loss every 10 epochs
            if epoch % 1 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")

#test
x = np.random.rand(3, 3)

target = np.random.rand(3, 2)
print('target:', target)
print('\n')

layer_one = Layer(3, 2, linear, d_linear)

layer_two = Layer(2, 2, linear, d_linear)

NN = NeuralNetwork()
NN.add_layer(layer_one)
NN.add_layer(layer_two)
NN.train(x, target, 10, 0.01, mean_squared_error, mean_squared_error_derivative)


target: [[0.81365885 0.14097459]
 [0.0717333  0.2486668 ]
 [0.47714637 0.60825001]]


Epoch 0, Loss: 0.20838199335487073
Epoch 1, Loss: 0.20804770311309087
Epoch 2, Loss: 0.20771434038052214
Epoch 3, Loss: 0.20738186734578348
Epoch 4, Loss: 0.20705024675041242
Epoch 5, Loss: 0.20671944187755642
Epoch 6, Loss: 0.20638941654096477
Epoch 7, Loss: 0.20606013507427212
Epoch 8, Loss: 0.20573156232056725
Epoch 9, Loss: 0.2054036636222385
